In [8]:
import torch
import numpy as np
# 单通道卷积做辅助
def Conv2d_single(X,K):
    h,w =K.shape #卷积核的高宽
    Y =torch.zeros(X.shape[0] - h + 1, X.shape[1] - w +1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            region = X[i:i + h, j:j + w]
            Y[i,j] = (region * K).sum()
    return Y
#多通道卷积    
class Conv2d:
    def __init__(self, in_c, out_c, k_size=3):
        #权重是每个输出通道有in_c个kernel，而每个kernel有3*3个参数
        self.weight = torch.randn(out_c, in_c, k_size, k_size)*0.1  # 权重怎么初始化？
        #bias每个输出通道一个即可
        self.bias = torch.randn(out_c)*0.1    # 偏置也很重要哦！
        
    def forward(self, X):
        in_c,H,W =X.shape
        out_c,_,k_h, k_w =self.weight.shape #一一对应
        H_out, W_out = H - k_h +1, W - k_w +1 #计算输出的高宽
        Y = torch.zeros((out_c,H_out,W_out)) #初始化

        for oc in range (out_c):# 遍历输出通道
            y= torch.zeros((H_out,W_out))
            for ic in range(in_c):# 遍历输入通道
                y+= Conv2d_single(X[ic],self.weight[oc,ic])
            Y[oc]= y+self.bias[oc]
        return Y


#测试一下
if __name__ == "__main__":
    X = torch.randn(3, 4, 5)   
    conv = Conv2d(in_c=3, out_c=6, k_size=3)
    Y = conv.forward(X)
    print(X.shape)
    print(Y.shape)

torch.Size([3, 4, 5])
torch.Size([6, 2, 3])


In [1]:
def Maxpool2d (X,pool_size= 2,stride =2):
    h,w = pool_size, pool_size
    #计算输出高宽
    out_h = (X.shape[0]-h+stride)/stride
    out_w = (X.shape[0]-w+stride)/stride
    Y = torch.zeros ((out_h, out_w))
    for i in range(out_h):
        for j in range(out_w):
            #卷积核扫描过的区域的高宽
            region = X[i*striede : i*stride + h, j*stride: j*stride +h ]
            Y[i,j] =region.max()#对此区域取最大值
    return Y 

def Avgpool2d (X,pool_size=2,stride=2):#同理
    h,w = pool_size,pool_size
    out_h = (X.shape[0]-h+ stride)/stride
    out_w = (X.shape[1]- h +stride)/stide
    Y = torch.zeros ((out_h, out_w))
    for i in range(out_h):
        for j in range (out_w):
            region = X[i*stride:i*stride+h, j*stride:j*stride +w]
            Y[i,j] = region.mean ()
    return Y

In [3]:
#动手实现batchnorm
import torch
from torch import nn

class BatchNorm(nn.Module):
    def __init__(self,num_features,num_dims):
        super().__init__()
        if num_dims == 2:#维度基本就是在2（全连接层）和4（卷积层）之间
            shape = (1,num_features)
        else:
            shape =(1,num_features,1,1)#BN对卷积层把通道当作特征学习参数
        #可学习的参数，对应的是新的均值和方差
        self.gamma = nn.Parameter(torch.ones(shpae))#新方差，所以不为0
        self.beta = nn.Paraeter(torch.zeros(shape))
        #定义可改变的均值和方差
        self.moving_mean= torch.zeros(shape)
        self.moving_var = torch.ones(shape)

    def forward(self,X) :
        if self.training:
            #定义全局的均值和方差
            mean = X.mean(dim=0, keepdim=Ture) #对当前小批量样本数据算
            var = ((X -mean)**2).mean(dim=0,keepdim=True)
            self.moving_mean = 0.9 * self.moving_mean + 0.1 * mean
            self.moving_var = 0.9 * self.moving_var + 0.1*var
        else:
            mean,var = self.moving_mean, self.moving_var

        X_hat = (X-mean)/ torch.sqrt(var + 1e-5) #加一个eps噪音
        return self.gamma * X_hat + self.beta
        #通过随机的缩放和偏差来控制模型复杂度
        #保证了既有随机性又不会变太剧烈（0.9的lr使self.moving变得比较慢）
        
    

In [7]:
import torch
from torch import nn
#首先构建一个vgg block
class vgg_block(nn.Module):
    #设定的参数包括输入输出通道和卷积层个数
    def __init__ (self,in_channels, out_channels,num_convs):
        super().__init__()
        layers =[]
        for i in range (num_convs):
            layers.append(nn.Conv2d(in_channels if i == 0 else out_channels,out_channels,kernel_size =3,padding = 1 ))
            layers.append(nn.ReLU(inplace=True)) #加一个非线性激活函数
        layers.append(nn.MaxPool2d(kernel_size=2,stride=2))#最大池化层
        self.block=nn.Sequential(*layers)

def forward(self,x):
    return self.block(x)
    


In [8]:
class vgg16(nn.Module):
    def __init__(self,num_classes=1000):#以imagenet为例，千分类
        super().__init__()
        #16=2+2+3+3+3(卷积层)+3(全连接)#设置每一个block的通道数和cv层个数
        self.features=nn.Sequential(vggblock(3,64,2),vggblock(64,128,2),vggblock(128,256,3),vggblock(256,512,3),vggblock(512,512,3))
        self.classifier = nn.Sequential(nn.Flatten(),nn.linear(512*7*7,4096),nn.ReLU(inplace=True),
                                        nn.Linear(4096,4096),nn.ReLU(inplace=True),
                                        nn.Linear(4096,num_classes)
                                       )#最后加3个全连接层
        def forward(self,x):
            x=self.features(x)
            x= self.classifier(x)
            return x
        #Pytorch会自动构建计算图，完成反向传播